In [ ]:
import pandas as pd
import numpy as np

# 1. Load Data
df = pd.read_csv("ANC and transfusion data/anc_45days.csv")

# Convert timestamps and handle timezones
df['charttime'] = pd.to_datetime(df['charttime'], errors='coerce').dt.tz_localize(None)
df['dischtime'] = pd.to_datetime(df['dischtime'], errors='coerce').dt.tz_localize(None)
df['admittime'] = pd.to_datetime(df['admittime'], errors='coerce').dt.tz_localize(None)

# 2. Filter out rows where valuenum is missing
df = df.dropna(subset=['valuenum'])

# 3. Create initial aplasia flag (1 if ANC <= 0.5, else 0)
df['aplasia_flag'] = np.where(df['valuenum'] <= 0.5, 1, 0)
df['event_date'] = df['charttime'].dt.floor('D')

# 4. Aggregate to daily level
daily = (
    df.groupby(['subject_id', 'chemo_hadm_id', 'dischtime', 'admittime', 'event_date'], as_index=False)
      .agg(
          aplasia_flag=('aplasia_flag', 'max'),
          daily_min_anc=('valuenum', 'min')  
      )
)

daily = daily.sort_values(['subject_id', 'event_date', 'dischtime'], ascending=[True, True, False])

# Remove duplicates per day (taking the first one)
daily_nodup = (
    daily.groupby(['subject_id', 'event_date'], as_index=False)
         .first()
)

# 5. Create calendar skeleton (0 to 45 days post discharge)
admissions = (
    daily_nodup[['subject_id', 'chemo_hadm_id', 'dischtime', 'admittime']]
    .dropna(subset=['dischtime'])
    .drop_duplicates()
    .sort_values(['subject_id', 'dischtime'])
    .reset_index(drop=True)
)

admissions['next_dischtime'] = admissions.groupby('subject_id')['dischtime'].shift(-1)

all_rows = []
for _, row in admissions.iterrows():
    sid = row['subject_id']
    hadm = row['chemo_hadm_id']
    disc_dt = pd.to_datetime(row['dischtime'])
    admit_dt = pd.to_datetime(row['admittime'])
    disc_date = disc_dt.normalize()
    dates = pd.date_range(disc_date, disc_date + pd.Timedelta(days=45), freq='D')

    tmp = pd.DataFrame({
        'subject_id': sid,
        'chemo_hadm_id': hadm,
        'dischtime': disc_dt,
        'admittime': admit_dt,
        'next_dischtime': row['next_dischtime'],
        'event_date': dates
    })
    all_rows.append(tmp)

full_cal = pd.concat(all_rows, ignore_index=True)

# Merge actual data onto skeleton
full = full_cal.merge(
    daily_nodup[['subject_id', 'chemo_hadm_id', 'event_date', 'aplasia_flag', 'daily_min_anc']], 
    on=['subject_id', 'chemo_hadm_id', 'event_date'],
    how='left'
)

# Filter out days before discharge
full['discharge_date'] = full['dischtime'].dt.normalize()
full['day_offset'] = (full['event_date'] - full['discharge_date']).dt.days
full = full[full['day_offset'] >= 0].reset_index(drop=True)

# 6. Forward Fill Logic (Fills gaps <= 7 days with 2)
def forward_fill_between_ones(g, max_gap_days=7):
    g = g.sort_values('event_date').copy().reset_index(drop=True)
    g['aplasia_flag_filled'] = g['aplasia_flag'].copy()

    ones_idx = g.index[g['aplasia_flag'] == 1].tolist()
    if len(ones_idx) < 2:
        return g

    for i in range(len(ones_idx) - 1):
        s_idx = ones_idx[i]
        t_idx = ones_idx[i + 1]

        s_date = g.at[s_idx, 'event_date']
        t_date = g.at[t_idx, 'event_date']
        gap_days = (t_date - s_date).days

        between_mask = (g['event_date'] > s_date) & (g['event_date'] < t_date)
        any_zero_between = g.loc[between_mask, 'aplasia_flag'].eq(0).any()
        
        if any_zero_between:
            continue

        if 1 <= gap_days <= max_gap_days:
            fill_mask = between_mask & g['aplasia_flag'].isna()
            g.loc[fill_mask, 'aplasia_flag_filled'] = 2
        else:
            continue

    return g

full = (
    full.groupby(['subject_id', 'chemo_hadm_id'], group_keys=False)
        .apply(lambda gg: forward_fill_between_ones(gg, max_gap_days=7))
        .reset_index(drop=True)
)

# 7. Episode Detection
def detect_episodes(g):
    g = g.sort_values('event_date').copy()
    
    
    g['is_aplasia'] = g['aplasia_flag_filled'].isin([1, 2])
    g['prev_is_aplasia'] = g['is_aplasia'].shift(1).fillna(False)
    
    g['episode_start'] = (g['is_aplasia'] & ~g['prev_is_aplasia']).astype(int)
    
    g['episode_id'] = g['episode_start'].cumsum()
    
    g.loc[~g['is_aplasia'], 'episode_id'] = 0
    
    return g

full_ep = (
    full.groupby(['subject_id', 'chemo_hadm_id'], group_keys=False)
        .apply(detect_episodes)
        .reset_index(drop=True)
)

aplasia_days = full_ep[full_ep['aplasia_flag_filled'].isin([1, 2])].copy()

episodes = (
    aplasia_days
    .groupby(['subject_id', 'chemo_hadm_id', 'episode_id'], as_index=False)
    .agg(
        start_date=('event_date', 'min'),
        end_date=('event_date', 'max'),
        duration_days=('event_date', lambda s: (s.max() - s.min()).days + 1),
        minimum_anc_across_episode=('daily_min_anc', 'min') 
    )
    .sort_values(['subject_id', 'chemo_hadm_id', 'episode_id'])
)

episodes = episodes.sort_values(['subject_id', 'start_date'], ascending=[True, True]).reset_index(drop=True)

# 9. Merge discharge info back
discharge_map = (
    daily_nodup[['subject_id', 'chemo_hadm_id', 'dischtime', 'admittime']]
    .dropna()
    .drop_duplicates()
)

episodes = episodes.merge(
    discharge_map,
    on=['subject_id', 'chemo_hadm_id'],
    how='left'
)


# Final Selection
episodes = episodes[[
    'subject_id', 'chemo_hadm_id', 'admittime', 'dischtime', 
    'episode_id', 'start_date', 'end_date', 'duration_days', 'minimum_anc_across_episode'
]]


/var/folders/t_/rl1rdcns0mz201yxx18dvx_r0000gn/T/ipykernel_78642/3870733461.py:113: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda gg: forward_fill_between_ones(gg, max_gap_days=7))
/var/folders/t_/rl1rdcns0mz201yxx18dvx_r0000gn/T/ipykernel_78642/3870733461.py:123: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  g['prev_is_aplasia'] = g['is_aplasia'].shift(1).fillna(False)
/var/folders/t_/rl1rdcns0mz201yxx18dvx_r0000gn/T/ipykernel_78642/3870733461.py:123: FutureWarning: Down

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colorbar import ColorbarBase
import matplotlib.cm as cm

patient_id = 16906565

COLOR_NOT_VALID  = '#444444'   
COLOR_NO_LAB     = '#d3d3d3'   
COLOR_NO_APLASIA = "#45acf5"   
COLOR_IMPUTED    = "#fff132"   

CODE_NOT_VALID   = 0
CODE_NO_LAB      = 1
CODE_NO_APLASIA  = 2
CODE_IMPUTED     = 3
CODE_HAS_ANC     = -1

full_local = full.copy()
full_local['event_date'] = pd.to_datetime(full_local['event_date']).dt.normalize()
full_local['dischtime'] = pd.to_datetime(full_local['dischtime']).dt.tz_localize(None)
full_local['admittime'] = pd.to_datetime(full_local['admittime']).dt.tz_localize(None)
full_local['discharge_date'] = full_local['dischtime'].dt.normalize()

# Get Admissions
patient_adm = (
    full_local[['subject_id', 'chemo_hadm_id', 'dischtime', 'admittime', 'discharge_date']]
    .drop_duplicates()
    .query("subject_id == @patient_id")
    .sort_values('dischtime')
    .reset_index(drop=True)
)

patient_adm['next_dischtime'] = patient_adm['dischtime'].shift(-1)

if patient_adm.empty:
    raise ValueError(f"No admissions found for patient {patient_id}")

patient_full_ep = full_ep.query("subject_id == @patient_id").copy()


lookup = {}
for row in patient_full_ep.itertuples():
    k = (int(row.chemo_hadm_id), pd.to_datetime(row.event_date).normalize())
    
    lookup[k] = (row.aplasia_flag_filled, row.daily_min_anc)


min_possible = patient_adm['discharge_date'].min()
max_possible = patient_full_ep['event_date'].max()
date_index = pd.date_range(min_possible, max_possible, freq='D')

rows = patient_adm.shape[0]
cols = len(date_index)


grid_cat = np.full((rows, cols), CODE_NOT_VALID, dtype=int)


grid_anc = np.full((rows, cols), np.nan, dtype=float)

for i, adm in patient_adm.iterrows():
    hadm = int(adm['chemo_hadm_id'])
    disc_date = adm['discharge_date']
    
    # Time Window Logic
    max_window_end = disc_date + pd.Timedelta(days=45)
    next_disc = adm['next_dischtime']
    
    if pd.notna(next_disc):
        next_disc_date = next_disc.normalize()
        cutoff_date = min(max_window_end, next_disc_date - pd.Timedelta(days=1))
    else:
        cutoff_date = max_window_end

    for j, d in enumerate(date_index):
        if d >= disc_date and d <= cutoff_date:
            key = (hadm, d)
            
            if key in lookup:
                flag, anc = lookup[key]
                
               
                if float(flag) == 2.0:
                    grid_cat[i, j] = CODE_IMPUTED
                
                
                elif pd.notna(anc):
                    if anc <= 0.5:
                        
                        grid_cat[i, j] = CODE_HAS_ANC 
                        
                        grid_anc[i, j] = anc
                    else:
                        
                        grid_cat[i, j] = CODE_NO_APLASIA
                
                
                else:
                    grid_cat[i, j] = CODE_NO_LAB
                    
               
            else:
                grid_cat[i, j] = CODE_NO_LAB


fig, ax = plt.subplots(figsize=(15, max(3, 0.8 * rows)))

# Create Coordinates
X, Y = np.meshgrid(np.arange(cols + 1), np.arange(rows + 1))


cmap_cat = ListedColormap([COLOR_NOT_VALID, COLOR_NO_LAB, COLOR_NO_APLASIA, COLOR_IMPUTED])
# Boundaries for 0, 1, 2, 3
norm_cat = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5], cmap_cat.N)


grid_cat_masked = np.ma.masked_where(grid_cat == CODE_HAS_ANC, grid_cat)

mesh_cat = ax.pcolormesh(X, Y, grid_cat_masked, cmap=cmap_cat, norm=norm_cat, 
                         edgecolors='black', linewidth=0.5)


reds_cmap = plt.cm.get_cmap('Reds')


custom_reds = reds_cmap(np.linspace(1.0, 0.20, 256))
cmap_anc = ListedColormap(custom_reds)


grid_anc_masked = np.ma.masked_invalid(grid_anc)

mesh_anc = ax.pcolormesh(X, Y, grid_anc_masked, cmap=cmap_anc, 
                         vmin=0.0, vmax=0.5, 
                         edgecolors='black', linewidth=0.5)

ax.invert_yaxis()


step = max(1, cols // 15)
xtick_positions = np.arange(0.5, cols, step)
xtick_labels = [d.strftime('%Y-%m-%d') for d in date_index[::step]]
ax.set_xticks(xtick_positions)
ax.set_xticklabels(xtick_labels, rotation=45, ha='right', fontsize=8)

# Y-Axis
adm_labels = (
    patient_adm['subject_id'].astype(str) + '_' +
    patient_adm['chemo_hadm_id'].astype(str) + ' (' +
    patient_adm['discharge_date'].dt.strftime('%Y-%m-%d') + ')'
)
ax.set_yticks(0.5 + np.arange(rows))
ax.set_yticklabels(adm_labels, fontsize=9)

# Grid Lines for Admissions
for i in range(1, len(patient_adm)):
    admit_dt = patient_adm.iloc[i]['admittime']
    if pd.notna(admit_dt):
        days_diff = (admit_dt.normalize() - date_index[0]).days
        if 0 <= days_diff < cols:
            ax.axvline(x=days_diff + 0.5, color='#FFFF00', linestyle='--', linewidth=2, alpha=0.8)

ax.set_title(f'Patient {patient_id} — Aplasia Heatmap')
ax.set_ylabel('Admissions')
ax.set_xlabel('Date')



# 1. Categorical Colorbar
cax1 = fig.add_axes([0.92, 0.45, 0.02, 0.4]) 
cb1 = ColorbarBase(cax1, cmap=cmap_cat, norm=norm_cat, ticks=[0, 1, 2, 3])
cb1.set_ticklabels(['Window Closed', 'No Lab', 'ANC > 0.5 (No Aplasia)', 'Filled ANC'])
cax1.set_title("Status", fontsize=9)

# 2. Continuous Colorbar (Red Gradient)
cax2 = fig.add_axes([0.92, 0.1, 0.02, 0.25]) 
cb2 = plt.colorbar(mesh_anc, cax=cax2, orientation='vertical')
cb2.set_label('ANC Value (Observed)')


cb2.set_ticks([0.0, 0.25, 0.5])
cb2.set_ticklabels(['0.0 (low ANC value)', '0.25', '0.5 (high ANC value)'])

plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.show()